In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import os 
import sys
import plotly.graph_objects as go
from os.path import join as pjoin
from scipy.stats import pearsonr, zscore, spearmanr
from tqdm import tqdm

sys.path.append('../..')
import circletrack_behavior as ctb
import circletrack_neural as ctn
import plotting_functions as pf
import place_cells as pc

/home/austinbaggetta/.conda/envs/calcium/lib/python3.10/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.2, the latest is 0.5.3.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [ ]:
## Set parameters
mouse = 'mc_EEG1_01'
root_path = os.path.abspath('../../../MultiCon_AfterHours/MultiCon_EEG1/minian_result/')
## Load separate datasets
short_sess_path = pjoin(root_path, f'{mouse}_twosess/minian')
long_sess_path = pjoin(root_path, f'{mouse}_minerva/minian')
## Set variables
ds_short = ctn.open_minian(short_sess_path)['S']
ds_long = ctn.open_minian(long_sess_path)['S']

In [ ]:
ds_short

In [ ]:
ds_long

In [ ]:
## Select a specific session
session_id = '0'
short_session = ds_short[:, ds_short['session_id'] == session_id]
long_session = ds_long[:, ds_long['session_id'] == session_id]

In [ ]:
## Compare each cell to every other cell to determine highest correlations
result = {'long_cell': [], 'short_cell': [], 'r': [], 'p': []}
for cell in tqdm(np.arange(0, long_session.shape[0])):
    for unit in np.arange(0, short_session.shape[0]):
        output = spearmanr(long_session.values[cell], short_session.values[unit])
        result['long_cell'].append(cell)
        result['short_cell'].append(unit)
        result['r'].append(output[0])
        result['p'].append(output[1])

In [ ]:
## Convert dictionary to dataframe
threshold = 0.5
df = pd.DataFrame(result)
df[df['r'] > threshold] ## find which cells have a higher correlation than the threshold given

In [ ]:
## Plot two cells from different sessions that have a high correlation
sampling_rate = 1/30
short_cell = 0
long_cell = 0
x_axis = np.arange(0, short_session.shape[1] * sampling_rate, sampling_rate)
fig = pf.custom_graph_template(x_title='Time (s)', y_title='Activation Strenght (a.u.)', width=800)
fig.add_trace(go.Scatter(x=x_axis, y=short_session.values[short_cell], mode='lines', line_color='turquoise', name='Short Session'))
fig.add_trace(go.Scatter(x=x_axis, y=long_session.values[long_cell], mode='lines', line_color='darkgrey', name='Long Session'))
fig.show()

In [ ]:
## Plot two cells from different sessions that have a high correlation
sampling_rate = 1/30
short_cell = 6
long_cell = 5
x_axis = np.arange(0, short_session.shape[1] * sampling_rate, sampling_rate)
fig = pf.custom_graph_template(x_title='Time (s)', y_title='Activation Strenght (a.u.)', width=800)
fig.add_trace(go.Scatter(x=x_axis, y=short_session.values[short_cell], mode='lines', line_color='turquoise', name='Short Session'))
fig.add_trace(go.Scatter(x=x_axis, y=long_session.values[long_cell], mode='lines', line_color='darkgrey', name='Long Session'))
fig.show()

In [ ]:
## Plot two cells from different sessions that have a high correlation
sampling_rate = 1/30
short_cell = 8
long_cell = 6
x_axis = np.arange(0, short_session.shape[1] * sampling_rate, sampling_rate)
fig = pf.custom_graph_template(x_title='Time (s)', y_title='Activation Strenght (a.u.)', width=800)
fig.add_trace(go.Scatter(x=x_axis, y=short_session.values[short_cell], mode='lines', line_color='turquoise', name='Short Session'))
fig.add_trace(go.Scatter(x=x_axis, y=long_session.values[long_cell], mode='lines', line_color='darkgrey', name='Long Session'))
fig.show()

In [ ]:
## Plot two cells from different sessions that have a high correlation
sampling_rate = 1/30
short_cell = 18
long_cell = 15
x_axis = np.arange(0, short_session.shape[1] * sampling_rate, sampling_rate)
fig = pf.custom_graph_template(x_title='Time (s)', y_title='Activation Strenght (a.u.)', width=800)
fig.add_trace(go.Scatter(x=x_axis, y=short_session.values[short_cell], mode='lines', line_color='turquoise', name='Short Session'))
fig.add_trace(go.Scatter(x=x_axis, y=long_session.values[long_cell], mode='lines', line_color='darkgrey', name='Long Session'))
fig.show()

In [ ]:
## Create histograms of spike counts for each session
short_spike_num = np.count_nonzero(short_session.values, axis=1)
long_spike_num = np.count_nonzero(long_session.values, axis=1)
fig = pf.custom_graph_template(x_title='Approximated Spike Count', y_title='Count', width=800)
fig.add_trace(go.Histogram(x=short_spike_num, nbinsx=60, marker_color='turquoise', name='Short Session'))
fig.add_trace(go.Histogram(x=long_spike_num, nbinsx=16, marker_color='darkgrey', name='Long Session'))
fig.update_layout(barmode='overlay')
fig.update_traces(opacity=0.75)
fig.show()

In [ ]:
## Population Vector Correlations of place fields
## Set parameters
dataset = {}
dpath = os.path.abspath('../../../MultiCon_AfterHours/MultiCon_EEG1/output/aligned_minian')
mouse_dict = {'mc_EEG1_01': ['A1', 'A2', 'A3', 'A4', 'A5']} ## set which sessions you will be comparing
alpha = 0.05
nbins = 20

for mouse in mouse_dict.keys():
    for session in mouse_dict[mouse]:
        file_name = '_'.join([mouse, session])
        da = xr.open_dataset(pjoin(dpath, f'{mouse}/{file_name}.nc'))['S'] ## select S matrix
        dataset[session] = da

In [ ]:
for session in dataset.keys():
    data = dataset[session]
    pf_data = pc.PlaceFields(x=data['x'].values,
                             y=data['y'].values,
                             t=data['behav_t'].values,
                             neural_data=data.values,
                             circular=True,
                             shuffle_test=True,
                             nbins=nbins)